# Web Scraping from IMDB Site:

 

Here I have built a step by step web scraper using Python.

For this study, I scrape data from IMDb’s “Top 1000” movies. The information I gather from each movie are:
* The title
* The year it was released
* How long the movie is
* IMDb’s rating of the movie
* The Metascore of the movie
* How many votes the movie got
* The U.S. gross earnings of the movie

Link of the IMDB website: https://www.imdb.com/search/title/?groups=top_1000&ref_=adv_prv


## Loading the libraries

In [1]:
import pandas as pd
import numpy as np

import requests
from requests import get
from bs4 import BeautifulSoup

from time import sleep
from random import randint

## Scraping from the web page

In [2]:
# Creating the lists we want to write into
titles = []
years = []
time = []
imdb_ratings = []
metascores = []
votes = []
us_gross = []

In [3]:
# Getting English translated titles from the movies
headers = {'Accept-Language': 'en-US, en;q=0.5'}

There are 1000 movies and each page has 50 movies listed.

So the first 50 movies' url: 
https://www.imdb.com/search/title/?groups=top_1000&ref_=adv_prv

Movies from 51 to 100:
https://www.imdb.com/search/title/?groups=top_1000&start=51&ref_=adv_nxt

Movies from 101 to 150:
https://www.imdb.com/search/title/?groups=top_1000&start=101&ref_=adv_nxt

...

In [4]:
pages = np.arange(1, 1001, 50)
pages

array([  1,  51, 101, 151, 201, 251, 301, 351, 401, 451, 501, 551, 601,
       651, 701, 751, 801, 851, 901, 951])

In [5]:
# Storing each of the urls of 50 movies 
for page in pages:
    # Getting the contents from the each url
    page = requests.get('https://www.imdb.com/search/title/?groups=top_1000&start=' + str(page) + '&ref_=adv_nxt', headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # Aiming the part of the html we want to get the information from
    movie_div = soup.find_all('div', class_='lister-item mode-advanced')
    
    # Controling the loop’s rate by pausing the execution of the loop for a specified amount of time
    # Waiting time between requests for a number between 2-10 seconds
    sleep(randint(2,10))
    
    for container in movie_div:
        # Scraping the movie's name
        name = container.h3.a.text
        titles.append(name)
        
        # Scraping the movie's year
        year = container.h3.find('span', class_='lister-item-year').text
        years.append(year)
        
        # Scraping the movie's length
        runtime = container.find('span', class_='runtime').text if container.p.find('span', class_='runtime') else '-'
        time.append(runtime)
        
        # Scraping the rating
        imdb = float(container.strong.text)
        imdb_ratings.append(imdb)
        
        # Scraping the metascore
        m_score = container.find('span', class_='metascore').text if container.find('span', class_='metascore') else '-'
        metascores.append(m_score)
        
        # Scraping votes and gross earnings
        nv = container.find_all('span', attrs={'name':'nv'})
        vote = nv[0].text
        votes.append(vote)
        grosses = nv[1].text if len(nv) > 1 else '-'
        us_gross.append(grosses)

## Creating the Data Set

In [6]:
movies = pd.DataFrame({'movie':titles,
                       'year':years,
                       'time_minute':time,
                       'imdb_rating':imdb_ratings,
                       'metascore':metascores,
                       'vote':votes,
                       'gross_earning':us_gross})

movies.head()

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Joker,(2019),122 min,8.5,59,"808,270",$335.45M
1,Parasite,(2019),132 min,8.6,96,"426,750",$53.37M
2,Knives Out,(2019),130 min,7.9,82,"319,809",$165.36M
3,Once Upon a Time... in Hollywood,(2019),161 min,7.7,83,"478,002",$142.50M
4,The Gentlemen,(2019),113 min,7.9,51,"146,955",-


In [7]:
movies.dtypes

movie             object
year              object
time_minute       object
imdb_rating      float64
metascore         object
vote              object
gross_earning     object
dtype: object

## Cleaning the Data Set

In [8]:
# Cleaning 'year' column
movies['year'] = movies['year'].str.extract('(\d+)').astype(int)
movies.head(3)

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Joker,2019,122 min,8.5,59,"808,270",$335.45M
1,Parasite,2019,132 min,8.6,96,"426,750",$53.37M
2,Knives Out,2019,130 min,7.9,82,"319,809",$165.36M


In [9]:
# Cleaning 'time_minute' column
movies['time_minute'] = movies['time_minute'].str.extract('(\d+)').astype(int)
movies.head(3)

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Joker,2019,122,8.5,59,"808,270",$335.45M
1,Parasite,2019,132,8.6,96,"426,750",$53.37M
2,Knives Out,2019,130,7.9,82,"319,809",$165.36M


In [10]:
# Cleaning 'metascore' column
movies['metascore'] = movies['metascore'].str.extract('(\d+)')
# convert it to float and if there are dashes turn it into NaN
movies['metascore'] = pd.to_numeric(movies['metascore'], errors='coerce')

In [11]:
# Cleaning 'vote' column
movies['vote'] = movies['vote'].str.replace(',', '').astype(int)
movies.head(3)

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Joker,2019,122,8.5,59.0,808270,$335.45M
1,Parasite,2019,132,8.6,96.0,426750,$53.37M
2,Knives Out,2019,130,7.9,82.0,319809,$165.36M


In [12]:
# Cleaning 'gross_earning' column
# left strip $ and right strip M 
movies['gross_earning'] = movies['gross_earning'].map(lambda x: x.lstrip('$').rstrip('M'))
# convert it to float and if there are dashes turn it into NaN
movies['gross_earning'] = pd.to_numeric(movies['gross_earning'], errors='coerce')
movies.head(3)

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Joker,2019,122,8.5,59.0,808270,335.45
1,Parasite,2019,132,8.6,96.0,426750,53.37
2,Knives Out,2019,130,7.9,82.0,319809,165.36


In [13]:
movies.dtypes

movie             object
year               int64
time_minute        int64
imdb_rating      float64
metascore        float64
vote               int64
gross_earning    float64
dtype: object

## Final Data Set

In [14]:
movies

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Joker,2019,122,8.5,59.0,808270,335.45
1,Parasite,2019,132,8.6,96.0,426750,53.37
2,Knives Out,2019,130,7.9,82.0,319809,165.36
3,Once Upon a Time... in Hollywood,2019,161,7.7,83.0,478002,142.50
4,The Gentlemen,2019,113,7.9,51.0,146955,NaN
...,...,...,...,...,...,...,...
995,"Carry On, Munna Bhai",2006,144,8.1,NaN,41268,2.22
996,And Now for Something Completely Different,1971,88,7.6,NaN,28234,NaN
997,Boy A,2007,102,7.6,75.0,36297,0.11
998,English Vinglish,2012,134,7.8,NaN,32246,1.67


## Saving the data to a csv file

In [15]:
movies.to_csv('movies.csv')